# Recommendation System
1. Content-based model<br>
2. Collaborative filtering model<br>
3. Matrix Factorization<br>
4. Deep model

1. Content-based model
- Ma trận user-item được xây dựng dựa theo "nội dung" của item mà không quan tâm mối liên hệ giữa các user
- item được phân loại theo từng cluster theo từng mục đích (nội dung, tags, ...)
- Xây dựng ma trận theo độ tương quan 
1.1 Utility matrix
- Là ma trận với index hàng là user và cột là item (NxM)
- Giá trị được sử dụng là điểm rating của từng user với một hay nhiều item. Điều này dẫn đến ma trận xây dựng được là ma trận thưa


(bổ sung)

In [8]:
import os
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.sparse.linalg import svds

from utils import train_test_split
from model_selection import fastSVD, NMF, SVD

sns.set(rc={'figure.figsize':(10, 6)})

In [2]:
sns.set(rc={'figure.figsize':(10, 6)})
seed = 123
np.random.seed(seed)

## 1. Load data

### Summary

```
ml-1m
├── users.dat
├── movies.dat
├── ratings.dat
└── README.txt
```

In [3]:
PATH = "ml-1m/"
ratings = pd.read_csv(os.path.join(PATH, 'ratings.dat'), sep='::', names=["userID", "movieID", "rating", "timestamp"], engine='python')
movies = pd.read_csv(os.path.join(PATH, 'movies.dat'), sep='::', names=["movieID", "title", "genres"], engine='python')
users = pd.read_csv(os.path.join(PATH, 'users.dat'), sep='::', names=["userID", "gender", "age", "occupation", "zipcode"], engine='python')

In [4]:
users

,userID,gender,age,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455
...,...,...,...,...,...
6035,6036,F,25,15,32603
6036,6037,F,45,1,76006
6037,6038,F,56,1,14706
6038,6039,F,45,0,01060


## 2. EDA

### 2.1 ratings.dat

In [ ]:
ratings.head()

In [ ]:
sns.distplot(ratings["rating"], kde=False)

In [ ]:
ratings['datetime'] = pd.to_datetime(ratings['timestamp'], unit='s')
ratings.drop('timestamp', axis=1, inplace=True)
sns.distplot(ratings['datetime'], kde=False)

In [ ]:
num_userId = len(ratings.userID.unique())
num_movieId = len(ratings.movieID.unique())
print(num_userId)
print(num_movieId)
print("Coverage : {:.4f}%".format(len(ratings) * 100 / (num_userId * num_movieId)))

### 2.2 movies.dat

In [ ]:
movies.head()

In [ ]:
movies_refine = movies.copy()
movies_refine['year'] = (movies_refine['title'].str.extract(r'(\d{4})')).astype('int32')
movies_refine = movies_refine.dropna()
movies_refine['title'] = (movies_refine['title'].str.extract(r'(^[^\(]+)'))[0]
movies_refine['genres'] = movies_refine['genres'].str.split('|')

movies_refine

In [ ]:
movies_refine.year.unique()
movies_refine.year.value_counts()

In [ ]:
genre_count = dict()
for index, series in movies_refine.iterrows():
    for genre in series['genres']:
        genre_count[genre] = genre_count.get(genre, 0) + 1
pprint(genre_count)

### 2.3 users.dat

In [ ]:
users.head()

In [ ]:
users.gender.value_counts()

In [ ]:
users.age.value_counts()

In [ ]:
len(users)

In [ ]:
len(ratings.userID.unique())

## 3. Matrix Factorization Collaborative Filtering

Giải quyết vấn đề về ma trận thưa
- Tính mean của giá trị theo hàng :
$E(x) = \sum_{i=1}^{m} v_i $
- Chuẩn hoá : trừ giá trị hàng cho mean của từng hàng

- Tính ma trận hiệp phương sai :
(công thức)

SVD:
Tách utility matrix thành 3 ma trận : user - strength, strength - strength, strength - item
Ví dụ:
$$\begin{bmatrix}
 3 &2 &2 \\
  2  &3 &-2 
\end{bmatrix}
=
\begin{bmatrix}
 1/\sqrt[]{2}  & 1/\sqrt[]{2} \\
  1/\sqrt[]{2}  & -1/\sqrt[]{2} 
\end{bmatrix}
.
\begin{bmatrix}
 5 &0 &0 \\
 0 &3 &0 
\end{bmatrix}
.
\begin{bmatrix}
 1/\sqrt[]{2}  & 1/\sqrt[]{2}  &0 \\
  1/\sqrt[]{18}  & -1/\sqrt[]{18} & -1/\sqrt[]{18} \\
2/3 & -2/3 & -1/3
\end{bmatrix} $$

Giả sử ma trận A(2x3) tương ứng với 2 user - 3 items , value là rating của mỗi user tương ứng với mỗi item:
- Ma trận U (2x2) : mỗi cột thể hiện độ "mạnh" tương ứng trị riêng của ma trận $\sum$
- Ma trận $\sum$ (2x3): là ma trận đường chéo với trị riêng giảm dần (Giá trị $\sigma _{1}$ là giá trị có độ mạnh )
- Ma trận V (3x3) : mỗi hàng thể hiện khả năng match giữ mỗi cluster của item và điểm rating của mỗi user

### 3.1 Low rank Approximation (Truncated SVD)

In [ ]:
data_mat = np.array(ratings.pivot(index = 'movieID', columns = 'userID', values = 'rating'))
data_mat = np.nan_to_num(data_mat)

In [ ]:
nor_data_mat = train - np.mean(train, axis = 0)
u, s, vT = svds(nor_data_mat, k = 50)  # 50 eigenvalues

In [ ]:
recon_data_mat = u.dot(np.diag(s)).dot(vT)
recon_data_mat

In [ ]:
def recommend_movies(pred_matrix, userID, num_recommendations):
    """Recommend movies based on reconstructed svd matrix
    Params:
     - pred_matrix (num_movies, num_users) : reconstructed matrix
     - userID (scalar)
     - num_recommendations (scalar)
    Outputs:
     - movies
    """
    sorted_predict_idx = np.argsort(pred_matrix[:, userID-1])[::-1]

    user_data = ratings[ratings.userID == (userID)]
    user_full = user_data.merge(movies_refine).sort_values(['rating'], ascending=False)
    # print(user_full)
    print('User {0} has already rated {1} movies.'.format(userID, len(user_full)))
    print('Recommending highest {0} predicted ratings movies not already rated.'.format(num_recommendations))

    recommendations = movies_refine.iloc[sorted_predict_idx, :][~movies_refine.movieID.isin(user_full.movieID)]

    return user_full, recommendations.head(num_recommendations)

In [ ]:
rated, _ = recommend_movies(recon_data_mat, 1310, 20)
_

### 3.2.1 FunkSVD (slow ver.)

In [ ]:
model_svd = SVD(ratings, K=10, lambd=.1, lr_rate=0.75, max_iter=100, verbose=True, user_based=1, use_biased=False)

### 3.2.2 FunkSVD (fast ver.)

In [5]:
X_train, X_valid, X_test = train_test_split(ratings, split_ratio=0.7, shuffle=True)

In [6]:
model_svd = fastSVD(K=15, lambd=0.06, lr_rate=0.004, max_iter=50)
model_svd.fit(X_train, X_valid, early_stopping=True, verbose=False, use_biased=True)

Load data & Preprocessing !
Epoch 1/50
Valid loss: 0.90869 --- Valid RMSE: 0.95325 --- Valid MAE: 0.76154
Epoch 2/50
Valid loss: 0.86726 --- Valid RMSE: 0.93127 --- Valid MAE: 0.74082
Epoch 3/50
Valid loss: 0.85143 --- Valid RMSE: 0.92273 --- Valid MAE: 0.73304
Epoch 4/50
Valid loss: 0.84337 --- Valid RMSE: 0.91835 --- Valid MAE: 0.72902
Epoch 5/50
Valid loss: 0.83863 --- Valid RMSE: 0.91577 --- Valid MAE: 0.72662
Epoch 6/50
Valid loss: 0.83558 --- Valid RMSE: 0.91410 --- Valid MAE: 0.72505
Epoch 7/50
Valid loss: 0.83348 --- Valid RMSE: 0.91295 --- Valid MAE: 0.72395
Epoch 8/50
Valid loss: 0.83192 --- Valid RMSE: 0.91210 --- Valid MAE: 0.72313
Epoch 9/50
Valid loss: 0.83069 --- Valid RMSE: 0.91142 --- Valid MAE: 0.72247
Epoch 10/50
Valid loss: 0.82964 --- Valid RMSE: 0.91085 --- Valid MAE: 0.72192
Epoch 11/50
Valid loss: 0.82866 --- Valid RMSE: 0.91031 --- Valid MAE: 0.72141
Epoch 12/50
Valid loss: 0.82766 --- Valid RMSE: 0.90976 --- Valid MAE: 0.72091
Epoch 13/50
Valid loss: 0.82654 -

In [ ]:
model_svd.predict(X_test)

### 3.3 Nonnegative Matrix Factorization

In [9]:
model_nmf = NMF(K=15, lambd_pu=0.4, lambd_qi=0.7, max_iter=10)

In [10]:
model_nmf.fit(X_train, X_valid, early_stopping=False, verbose=True)

Load data & Preprocessing !
Epoch 1/10
Epoch 2/10
Epoch 3/10
Epoch 4/10
Epoch 5/10
Epoch 6/10
Epoch 7/10
Epoch 8/10
Epoch 9/10
Epoch 10/10
Valid loss: 554988.3837439748 --- Valid RMSE: 744.9754249262018 --- Valid MAE: 10.595040921077269


## 4 Deep learning

### 4.1 Simple deep model

### 4.2 Neural Collaborative Filtering